In [53]:
!pip install tflearn

    100% |████████████████████████████████| 102kB 1.2MB/s a 0:00:011
  Running setup.py bdist_wheel for tflearn ... done
  Stored in directory: /home/arunima/.cache/pip/wheels/fb/06/72/0478c938ca315c6fddcce8233b80ec91a115ce4496a27e8c90
Successfully built tflearn
You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import pandas as pd
import gzip
import time
import tensorflow as tf
import tflearn
# Install a few python packages using pip
from common import utils
utils.require_package('nltk')
utils.require_package("wget")      # for fetching dataset
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

/home/arunima/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# Standard python helper libraries.
from __future__ import print_function
from __future__ import division
import os, sys, time
import collections
import itertools

# Numerical manipulation libraries.
import numpy as np
from scipy import stats, optimize

import nltk
nltk.download('punkt')
from nltk import word_tokenize

# Helper libraries
from common import utils, vocabulary, tf_embed_viz, glove_helper

[nltk_data] Downloading package punkt to /home/arunima/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#Function to read the amazon review data files
def parse(path):
  print('start parse')
  start_parse = time.time()
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)
  end_parse = time.time()
  print('end parse with time for parse',end_parse - start_parse)

def getDF(path):
  print('start getDF')
  start = time.time()
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  print('end getDF')
  end = time.time()
  print('time taken to load data = ',end-start)
  return pd.DataFrame.from_dict(df, orient='index')
#df = getDF('reviews_Toys_and_Games.json.gz') #old def function corresponding to the step bt step vectorization

In [4]:
#Using pretrained GLove embeddings
hands = glove_helper.Hands(ndim=50)  # 50, 100, 200, 300 dim are available
hands.shape
print(hands.shape[1])

Loading vectors from data/glove/glove.6B.zip
Parsing file: data/glove/glove.6B.zip:glove.6B.50d.txt
Found 400,000 words.
Parsing vectors... Done! (W.shape = (400003, 50))
50


In [5]:
df_toys = getDF('reviews_Toys_and_Games.json.gz')

start getDF
start parse
end parse with time for parse 111.86842513084412
end getDF
time taken to load data =  111.86901903152466


In [ ]:
df_vid = getDF('reviews_Video_Games.json.gz')

In [7]:
df_aut = getDF('reviews_Automotive.json.gz')

start getDF
start parse
end parse with time for parse 76.23472046852112
end getDF
time taken to load data =  76.2351188659668


In [8]:
df_hnk = getDF('reviews_Home_and_Kitchen.json.gz')

start getDF
start parse
end parse with time for parse 277.64442777633667
end getDF
time taken to load data =  277.64523220062256


In [9]:
#Looking at a few examples of review text
print('Toys reviews examples\n')
for i in range(1):
    print(df_toys['reviewerID'].iloc[i])
    print(df_toys['reviewText'].iloc[i])

print('\n Video games reviews examples\n')
for i in range(1):
    print(df_vid['reviewerID'].iloc[i])
    print(df_vid['reviewText'].iloc[i])
    
print('\n Automobile reviews examples\n')
for i in range(1):
    print(df_aut['reviewerID'].iloc[i])
    print(df_aut['reviewText'].iloc[i])
    
print('\n Home and Kitchen reviews examples\n')
for i in range(1):
    print(df_hnk['reviewerID'].iloc[i])
    print(df_hnk['reviewText'].iloc[i])

Toys reviews examples

AMEVO2LY6VEJA
Great product, thank you! Our son loved the puzzles.  They have large pieces yet they are still challenging for a 4 year old.

 Video games reviews examples

AB9S9279OZ3QO
I haven't gotten around to playing the campaign but the multiplayer is solid and pretty fun. Includes Zero Dark Thirty pack, an Online Pass, and the all powerful Battlefield 4 Beta access.

 Automobile reviews examples

A108J5O7DG2WIM
I loved the look and the great improvement at night drivingA bit expensive but work great.Installation took me about 3 hours.

 Home and Kitchen reviews examples

A210NOCSTBT4OD
Have you ever thought about how you met your best friend? Was it normal, or was it wacky - like how Elias met Shohei? Pulling a boa constrictor snake named Mathilda out of your backpack can make a remarkable first impression! This book is about three best friends Elias, Honoria, and Shohei, who are united against "That Which Is The Peshtigo School". Their goal is to make it t

In [17]:
#Create train,dev,test split
from sklearn.model_selection import train_test_split
train_toys,devtest = train_test_split(df_toys, test_size=0.4, random_state=42)
dev_toys,test_toys = train_test_split(devtest,test_size = 0.5, random_state=42)
print('Toy reviews train, dev and test set dataframe shape:',train_toys.shape,dev_toys.shape,test_toys.shape)

#For Video games reviews
# train_vid,devtest = train_test_split(df_vid, test_size=0.4)
# dev_vid,test_vid = train_test_split(devtest,test_size = 0.5)
# print('Video games reviews train, dev and test set dataframe shape:',train_vid.shape,dev_vid.shape,test_vid.shape)

#For Auto reviews
# train_aut,devtest = train_test_split(df_aut, test_size=0.4)
# dev_aut,test_aut = train_test_split(devtest,test_size = 0.5)
# print('Auto reviews train, dev and test set dataframe shape:',train_aut.shape,dev_aut.shape,test_aut.shape)

#For Home and Kitchen reviews
# train_hnk,devtest = train_test_split(df_hnk, test_size=0.4)
# dev_hnk,test_hnk = train_test_split(devtest,test_size = 0.5)
# print('Home and Kitchen reviews train, dev and test set dataframe shape:',train_hnk.shape,dev_hnk.shape,test_hnk.shape)

Toy reviews train, dev and test set dataframe shape: (1351662, 9) (450554, 9) (450555, 9)


In [8]:
#Function to create a smaller sized train and dev data set. Enables testing accuracy for different sizes.
#Also binarizes the labels. Ratings of 1,2 and to 0; Ratings of 4,5 to 1.

def set_df_size(size,data_train,data_dev):
    size_train = size
    len_max_train = data_train[data_train.overall!=3].shape[0] #max possible length of train data set taking out the 3 ratings.
    #print("Number of reviews with ratings != 3 in train set",len_max_train)
    temp_size_train = min(len_max_train,size_train)

    len_max_dev = data_dev[data_dev.overall!=3].shape[0]
    #print("Number of reviews with ratings != 3 in dev set",len_max_dev)
    temp_size_dev = min(len_max_dev,int(0.3*temp_size_train)) #making the dev set about 0.3 times the train set.

    temp_train_data = data_train[data_train.overall != 3][:temp_size_train]
    #print('Size of train data',temp_train_data.shape)
    #print(temp_train_data.groupby('overall').count())
    #print(temp_train_toys[:5])

    temp_dev_data = data_dev[data_dev.overall!=3][:temp_size_dev]
    #print('Size of dev data',temp_dev_data.shape)
    #print(temp_dev_data.groupby('overall').count())
    #print(temp_dev_data[:2])
    
    #Binarize ratings
    temp_train_y = np.zeros(temp_size_train)
    temp_train_y[temp_train_data.overall > 3] = 1
    temp_dev_y = np.zeros(temp_size_dev)
    temp_dev_y[temp_dev_data.overall>3] = 1
    #print('binarized y shape',temp_train_y.shape,temp_dev_y.shape)
    #print(temp_dev_y[:20],data_dev.overall[:20])
    return temp_train_data,temp_dev_data,temp_train_y,temp_dev_y

In [18]:
list_df = ['toys','vid','aut','hnk'] #list of keys that refer to each dataframe. Adding a new dataframe would require updating this list
dict_train_df = {} #Dict to store train input data frame for each domain, can be accessed by using domain name as key
dict_dev_df = {} #Dict to store dev input data frame for each domain, can be accessed by using domain name as key
dict_train_y = {} #Dict to store binarized train data label for each domain
dict_dev_y = {} #Dict to store binarized dev data label for each domain
#print(len(dict_train_df))

def create_sized_data(size = 10000):
    size_train = size #Set size of train set here. This is a hyperparameter.
    key = list_df[0]
    #print('Toys reviews\n')
    dict_train_df[key], dict_dev_df[key], dict_train_y[key], dict_dev_y[key] = set_df_size(size_train,train_toys,dev_toys)
#     #print('\n Video games reviews\n')
#     key = list_df[1]
#     dict_train_df[key], dict_dev_df[key], dict_train_y[key], dict_dev_y[key] = set_df_size(size_train,train_vid,dev_vid)
#     #print('\n Auto reviews\n')
#     key = list_df[2]
#     dict_train_df[key], dict_dev_df[key], dict_train_y[key], dict_dev_y[key] = set_df_size(size_train,train_aut,dev_aut)
#     #print('\n Home and Kitchen reviews\n')
#     key = list_df[3]
#     dict_train_df[key], dict_dev_df[key], dict_train_y[key], dict_dev_y[key] = set_df_size(size_train,train_hnk,dev_hnk)
    
create_sized_data()
#print(len(dict_train_df))

In [11]:
# Hyperparameters
max_length = 200


In [ ]:
#Note : This function seems to be deprecated. Another function I ran into 
# tflearn.data_utils.VocabularyProcessor (max_document_length, min_frequency=0, vocabulary=None, tokenizer_fn=None)
#coded in the next cell
# vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(max_length)

# def process_inputs(key, vocab_processor):
    
#     # For simplicity, we call our features x and our outputs y
#     start_vectorize = time.time()
#     x_train = dict_train_df[key].reviewText
#     y_train = dict_train_y[key]
#     x_dev = dict_dev_df[key].reviewText
#     y_dev = dict_dev_y[key]
#     print(x_train.shape)
    
#     # Train the vocab_processor from the training set
#     x_train = vocab_processor.fit_transform(x_train)
#     # Transform our test set with the vocabulary processor
#     x_dev = vocab_processor.transform(x_dev)

#     # We need these to be np.arrays instead of generators
#     x_train = np.array(list(x_train))
#     print(x_train.shape)
#     x_dev = np.array(list(x_dev))
#     y_train = np.array(y_train).astype(int)
#     y_dev = np.array(y_dev).astype(int)
    
#     y_train = tf.expand_dims(y_train,1)
#     y_dev = tf.expand_dims(y_dev,1)
#     print('y train shape',y_train.shape)

#     V = len(vocab_processor.vocabulary_)
#     print('Total words: %d' % V)
#     end_vectorize = time.time()
#     print('Time taken to vectorize %d size dataframe'%x_train.shape[0],end_vectorize-start_vectorize)

#     # Return the transformed data and the number of words
#     return x_train, y_train, x_dev, y_dev, V

# x_train, y_train, x_dev, y_dev, V = process_inputs('toys',vocab_processor)

# #Print a few examples for viewing
# print('sample review',dict_train_df['toys']['reviewText'].iloc[3],'\n')
# print('corresponding ids\n',x_train[3])
# print('sample review',dict_dev_df['toys']['reviewText'].iloc[3],'\n')
# print('corresponding ids\n',x_dev[3])

In [19]:
vocab_processor = tflearn.data_utils.VocabularyProcessor(max_length, min_frequency=0)
#Note : This function seems to be deprecated. Another function I ran into 
# tflearn.data_utils.VocabularyProcessor (max_document_length, min_frequency=3, vocabulary=None, tokenizer_fn=None)

def process_inputs(key, vocab_processor):
    
    # For simplicity, we call our features x and our outputs y
    start_vectorize = time.time()
    x_train = dict_train_df[key].reviewText
    y_train = dict_train_y[key]
    x_dev = dict_dev_df[key].reviewText
    y_dev = dict_dev_y[key]
    print(x_train.shape)
    
    # Train the vocab_processor from the training set
    x_train = vocab_processor.fit_transform(x_train)
    # Transform our test set with the vocabulary processor
    x_dev = vocab_processor.transform(x_dev)

    # We need these to be np.arrays instead of generators
    x_train = np.array(list(x_train))
    print(x_train.shape)
    x_dev = np.array(list(x_dev))
    y_train = np.array(y_train).astype(int)
    y_dev = np.array(y_dev).astype(int)
    
#     y_train = tf.expand_dims(y_train,1)
#     y_dev = tf.expand_dims(y_dev,1)
    print('y train shape',y_train.shape)

    V = len(vocab_processor.vocabulary_)
    print('Total words: %d' % V)
    end_vectorize = time.time()
    print('Time taken to vectorize %d size dataframe'%x_train.shape[0],end_vectorize-start_vectorize)

    # Return the transformed data and the number of words
    return x_train, y_train, x_dev, y_dev, V

x_train, y_train, x_dev, y_dev, V = process_inputs('toys',vocab_processor)

#Print a few examples for viewing
print('sample review',dict_train_df['toys']['reviewText'].iloc[3],'\n')
print('corresponding ids\n',x_train[3])
print('sample review',dict_dev_df['toys']['reviewText'].iloc[3],'\n')
print('corresponding ids\n',x_dev[3])

(10000,)
(10000, 200)
y train shape (10000,)
Total words: 28336
Time taken to vectorize 10000 size dataframe 1.6631669998168945
sample review It's just the model I was hoping for and more. It's challenging and has given me something more to learn in developing my model building skills. Revell as always makes good models to build! 

corresponding ids
 [58  9 48 59 33 50 60 39  4 61 58 62  4 63 64 65 66 61 67 68 69 70 51 59 71
 72 73 74 75 76 77 78 67 79  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
sample review Excellent example of the

In [22]:
print(y_train.shape)
print(y_dev.shape)
print(np.mean(y_train))

(10000,)
(3000,)
0.8585


In [33]:
class TextCNN(object):

    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """

    def __init__( self, sequence_length, num_classes, vocab_size, learning_rate, momentum, embedding_size, 
                 gl_embed, filter_sizes, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.int32, [None], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer

        with tf.name_scope("embedding"):
            #self.W = tf.get_variable("W_in",[vocab_size, embedding_size],initializer =tf.random_uniform_initializer(0,1)) #from wildML
            self.W=tf.get_variable(name="embedding_",shape=gl_embed.shape,
                                       initializer=tf.constant_initializer(gl_embed),trainable=True)
            self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)
            #print('embedded_chars',self.embedded_chars.get_shape())
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)
            #print('embedded_chars_expanded',self.embedded_chars_expanded.get_shape())

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []

        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):

                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                #W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                Wname = "w_%d"%filter_size
                W = tf.get_variable(Wname, shape = filter_shape, initializer = tf.contrib.layers.xavier_initializer())
                b = tf.Variable(tf.constant(0.0, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d( self.embedded_chars_expanded, W, strides=[1, 1, 1, 1], padding="VALID", name="conv")

                # Apply nonlinearity
                conv+= b
                h = tf.nn.relu(conv, name="relu")
                #print('h',h.get_shape())

                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(h, ksize=[1, sequence_length - filter_size + 1, 1, 1], strides=[1, 1, 1, 1],
                    padding='VALID', name="pool")
                #print('pooled',pooled.get_shape())
                pooled_outputs.append(pooled)
                #print('pooled_outputs',type(pooled_outputs))
                #print('pooled_outputs as array',type(np.array(pooled_outputs)),np.array(pooled_outputs).shape)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3)
        #print('h_pool',self.h_pool.get_shape())
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])
        #print('h_pool_flat',self.h_pool_flat.get_shape())
        
        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable("W", shape=[num_filters_total, num_classes],initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.0, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            #print('self.scores',self.scores.get_shape())
            self.predictions = tf.argmax(self.scores, 1, name="predictions")
            #print('self.predictions',self.predictions.get_shape())
            
        # Calculate mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss
            #self.loss = tf.losses.mean_squared_error(self.input_y, self.scores)

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(tf.cast(self.predictions,tf.int32), self.input_y)
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")
#             correct_pred = tf.equal(tf.cast(tf.round(self.scores), tf.int32), self.input_y)
#             self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
            
        with tf.name_scope('train'):
            #self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.loss)
            self.optimizer = tf.train.MomentumOptimizer(learning_rate = learning_rate,momentum=momentum,use_nesterov=True).minimize(self.loss)

In [15]:
def batch_generator(ids, labels, batch_size=100):
            #ids is input, X_train
            #need to fix this to shuffle between epochs
    
            n_batches = len(ids)//batch_size
            ids, labels = ids[:n_batches*batch_size], labels[:n_batches*batch_size]
    
            for ii in range(0, len(ids), batch_size):
                yield ids[ii:ii+batch_size], labels[ii:ii+batch_size]

# for ii, (x, y) in enumerate(batch_generator(x_train, y_train, batch_size), 1):
#     print(ii,type(x),x.shape,type(np.array(y)),y.shape)

In [34]:
#Actual training loop:

#embed_dim = 50 #use when not using pre-trained embeddings
embed_dim = hands.shape[1]
filter_sizes= [3,4,5]
num_filters = 128
l2_reg_lambda = 0
learning_rate = 0.01
momentum = 0.9
keep_prob = 0.8
evaluate_train = 2 # of epochs at which to print test accuracy
evaluate_dev = 5 # of epochs at which to estimate and print dev accuracy
time_print = 10 # of epochs at which to print time taken
num_classes = 2
num_epochs = 100
num_checkpoints = 1
batch_size = 64

out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", "cnn"))
print("Writing to {}\n".format(out_dir))

with tf.Graph().as_default():

    with tf.Session() as sess:
        
        cnn = TextCNN(sequence_length=x_train.shape[1], num_classes=num_classes, vocab_size=V, learning_rate = learning_rate,
                        momentum = momentum, embedding_size=embed_dim, gl_embed = hands.W, filter_sizes= filter_sizes, 
                      num_filters=num_filters, l2_reg_lambda=l2_reg_lambda)
        sess.run(tf.global_variables_initializer())
        print('completed cnn creation')

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)

        # Write vocabulary
        ## vocab_processor.save(os.path.join(out_dir, "vocab"))

        print('# batches =', len(x_train)//batch_size)
        start = time.time()
        for e in range(num_epochs):
                    
            #sum_scores = np.zeros((batch_size*(len(x_train)//batch_size),1))
            total_loss = 0
            total_acc = 0
            for i, (x, y) in enumerate(batch_generator(x_train, y_train, batch_size), 1):
                feed = {cnn.input_x: x, cnn.input_y: y, cnn.dropout_keep_prob: keep_prob}
                _, scores, loss, accuracy = sess.run([cnn.optimizer,cnn.scores,cnn.loss, cnn.accuracy],feed_dict = feed)
                total_loss += loss*len(x)
                total_acc += accuracy*len(x)
                
                #sum_scores[i*batch_size:(i+1)*batch_size,:] = scores
                #print(np.mean(sum_scores))
                #time_str = datetime.datetime.now().isoformat()
            if e%evaluate_train==0:
                avg_loss = total_loss/(batch_size*(len(x_train)//batch_size))
                avg_acc = total_acc/(batch_size*(len(x_train)//batch_size))
                print("Train epoch {}, average loss {:g}, average accuracy {:g},".format(e, avg_loss, avg_acc))
                #print('average',np.mean(scores))

            if e%evaluate_dev==0:
                total_loss = 0
                total_acc = 0
                for ii, (x, y) in enumerate(batch_generator(x_dev, y_dev, batch_size), 1):
                    feed_dict = {cnn.input_x: x, cnn.input_y: y, cnn.dropout_keep_prob: 1.0}
                    loss, accuracy = sess.run([cnn.loss, cnn.accuracy],feed_dict)
                    total_loss += loss*len(x)
                    total_acc += accuracy*len(x)
                avg_loss = total_loss/(batch_size*(len(x_dev)//batch_size))
                avg_acc = total_acc/(batch_size*(len(x_dev)//batch_size))
                #time_str = datetime.datetime.now().isoformat()
                print("\tDev epoch {}, average loss {:g}, average accuracy {:g},".format(e, avg_loss, avg_acc))
            if e%time_print == 0:
                end = time.time()
                print("\t\tTime taken for",e,"epochs = ", end-start)

Writing to /home/arunima/fproject/final_project/runs/cnn

completed cnn creation
# batches = 156
Train epoch 0, average loss 0.413483, average accuracy 0.853265,
	Dev epoch 0, average loss 0.382564, average accuracy 0.857337,
		Time taken for 0 epochs =  35.52638792991638
Train epoch 2, average loss 0.329666, average accuracy 0.869792,
Train epoch 4, average loss 0.266951, average accuracy 0.892228,
	Dev epoch 5, average loss 0.298188, average accuracy 0.886209,
Train epoch 6, average loss 0.206666, average accuracy 0.918369,
Train epoch 8, average loss 0.151604, average accuracy 0.942909,
Train epoch 10, average loss 0.103298, average accuracy 0.96855,
	Dev epoch 10, average loss 0.323083, average accuracy 0.888247,
		Time taken for 10 epochs =  374.96409010887146
Train epoch 12, average loss 0.0781583, average accuracy 0.976663,
Train epoch 14, average loss 0.0526756, average accuracy 0.988181,
	Dev epoch 15, average loss 0.321952, average accuracy 0.890965,
Train epoch 16, average l

## KEEPING TRACK OF RESULTS FROM DIFFERENT RUNS
#### Number samples = 10000, Number batches = 156, Without pre-trained embeddings, no dropout
Train epoch 0, loss 0.357085, average loss 0.440941, acc 0.84375, average acc 0.845152,
	Dev epoch 0, loss 0.494501, average loss 0.394619, acc 0.78125, average acc 0.854959,
		Time taken for 0 epochs =  36.18872332572937
Train epoch 2, loss 0.25934, average loss 0.335786, acc 0.875, average acc 0.862079,
Train epoch 4, loss 0.214457, average loss 0.263726, acc 0.875, average acc 0.891827,
Train epoch 6, loss 0.161232, average loss 0.194851, acc 0.890625, average acc 0.92528,
Train epoch 8, loss 0.0968393, average loss 0.132971, acc 0.984375, average acc 0.958133,
Train epoch 10, loss 0.0598382, average loss 0.0935878, acc 1, average acc 0.978766,
	Dev epoch 10, loss 0.318434, average loss 0.279314, acc 0.875, average acc 0.891304,
		Time taken for 10 epochs =  366.54717350006104
Train epoch 12, loss 0.0432213, average loss 0.089715, acc 1, average acc 0.979768,
Train epoch 14, loss 0.0724975, average loss 0.299487, acc 1, average acc 0.957933,
Train epoch 16, loss 0.0388074, average loss 0.0520482, acc 1, average acc 0.991987,
Train epoch 18, loss 0.0239645, average loss 0.0351604, acc 1, average acc 0.997196,
Train epoch 20, loss 0.0157139, average loss 0.0272624, acc 1, average acc 0.996595,
	Dev epoch 20, loss 0.304895, average loss 0.283551, acc 0.921875, average acc 0.902853,
		Time taken for 20 epochs =  697.680163860321
Train epoch 22, loss 0.0131277, average loss 0.017179, acc 1, average acc 0.999299,
Train epoch 24, loss 0.0104588, average loss 0.0121853, acc 1, average acc 0.9999,
Train epoch 26, loss 0.0072446, average loss 0.00969622, acc 1, average acc 0.9999,
Train epoch 28, loss 0.00628954, average loss 0.00805781, acc 1, average acc 0.9999,
Train epoch 30, loss 0.00585206, average loss 0.00689346, acc 1, average acc 0.9999,
	Dev epoch 30, loss 0.37002, average loss 0.327092, acc 0.875, average acc 0.902514,
		Time taken for 30 epochs =  1029.1201057434082
Train epoch 32, loss 0.00559655, average loss 0.00594074, acc 1, average acc 0.9999,
Train epoch 34, loss 0.0053231, average loss 0.00518374, acc 1, average acc 1,
Train epoch 36, loss 0.00504235, average loss 0.00461033, acc 1, average acc 1,
Train epoch 38, loss 0.00477842, average loss 0.00416377, acc 1, average acc 1,
Train epoch 40, loss 0.00451324, average loss 0.00380536, acc 1, average acc 1,
	Dev epoch 40, loss 0.451459, average loss 0.382435, acc 0.859375, average acc 0.899796,
		Time taken for 40 epochs =  1360.2103555202484
        
        
#### Number samples = 10000, Number batches = 156, With pre-trained embeddings(Trainable = False), dropout = 0.8
Train epoch 0, average loss 0.819034, average acc 0.802784,
	Dev epoch 0, average loss 0.415172, average acc 0.853601,
		Time taken for 0 epochs =  35.559093713760376
Train epoch 2, average loss 0.403757, average acc 0.841046,
Train epoch 4, average loss 0.340479, average acc 0.860777,
	Dev epoch 5, average loss 0.329067, average acc 0.867188,
Train epoch 6, average loss 0.289147, average acc 0.882312,
Train epoch 8, average loss 0.237817, average acc 0.904948,
Train epoch 10, average loss 0.194272, average acc 0.923978,
	Dev epoch 10, average loss 0.330927, average acc 0.876698,
		Time taken for 10 epochs =  363.92726016044617
Train epoch 12, average loss 0.149883, average acc 0.940405,
Train epoch 14, average loss 0.128152, average acc 0.951322,
	Dev epoch 15, average loss 0.349508, average acc 0.877717,
Train epoch 16, average loss 0.101319, average acc 0.961639,
Train epoch 18, average loss 0.079585, average acc 0.970052,
Train epoch 20, average loss 0.0705579, average acc 0.97516,
	Dev epoch 20, average loss 0.364253, average acc 0.878057,
		Time taken for 20 epochs =  692.3398864269257
Train epoch 22, average loss 0.0631964, average acc 0.978466,
Train epoch 24, average loss 0.0484077, average acc 0.984876,
	Dev epoch 25, average loss 0.435054, average acc 0.877717,
Train epoch 26, average loss 0.0433892, average acc 0.985377,
Train epoch 28, average loss 0.0368327, average acc 0.988381,
Train epoch 30, average loss 0.0308169, average acc 0.990385,
	Dev epoch 30, average loss 0.570798, average acc 0.875679,
		Time taken for 30 epochs =  1052.273297548294
Train epoch 32, average loss 0.0291807, average acc 0.991086,
Train epoch 34, average loss 0.0271599, average acc 0.991987,
	Dev epoch 35, average loss 0.661539, average acc 0.87534,
Train epoch 36, average loss 0.029594, average acc 0.991486,
Train epoch 38, average loss 0.0236557, average acc 0.99359,
Train epoch 40, average loss 0.018746, average acc 0.995292,
	Dev epoch 40, average loss 0.506544, average acc 0.878397,
		Time taken for 40 epochs =  1466.0729427337646
        
        
#### Changes. Changed convolutional layer weights to xavier initialization. Added random see = 42 to train-test split. Set min_df to 3 in vectorizer to get rid of misspelled / very rare words. Dropped learning rate initial to 0.007

Train epoch 0, average loss 0.399899, average accuracy 0.858273,
	Dev epoch 0, average loss 0.374428, average accuracy 0.857337,
		Time taken for 0 epochs =  34.83389401435852
Train epoch 2, average loss 0.326706, average accuracy 0.869391,
Train epoch 4, average loss 0.269826, average accuracy 0.891126,
	Dev epoch 5, average loss 0.303238, average accuracy 0.88519,
Train epoch 6, average loss 0.219368, average accuracy 0.911659,
Train epoch 8, average loss 0.171234, average accuracy 0.935296,
Train epoch 10, average loss 0.13296, average accuracy 0.953325,
	Dev epoch 10, average loss 0.293018, average accuracy 0.887568,
		Time taken for 10 epochs =  370.47870922088623
Train epoch 12, average loss 0.100562, average accuracy 0.967348,
Train epoch 14, average loss 0.0793127, average accuracy 0.977063,
	Dev epoch 15, average loss 0.320119, average accuracy 0.886209,
Train epoch 16, average loss 0.0582729, average accuracy 0.988482,
Train epoch 18, average loss 0.0456755, average accuracy 0.990385,
Train epoch 20, average loss 0.0405185, average accuracy 0.992788,
	Dev epoch 20, average loss 0.321604, average accuracy 0.886889,
		Time taken for 20 epochs =  705.0958936214447
Train epoch 22, average loss 0.0351258, average accuracy 0.993089,
Train epoch 24, average loss 0.0270392, average accuracy 0.996194,
	Dev epoch 25, average loss 0.399808, average accuracy 0.884171,
Train epoch 26, average loss 0.0262923, average accuracy 0.995994,
Train epoch 28, average loss 0.0242657, average accuracy 0.995994,
Train epoch 30, average loss 0.0208821, average accuracy 0.996394,
	Dev epoch 30, average loss 0.413923, average accuracy 0.886889,
		Time taken for 30 epochs =  1039.4113600254059
Train epoch 32, average loss 0.017492, average accuracy 0.997696,
Train epoch 34, average loss 0.0146527, average accuracy 0.998097,
	Dev epoch 35, average loss 0.386267, average accuracy 0.884851,
Train epoch 36, average loss 0.0168233, average accuracy 0.997396,
Train epoch 38, average loss 0.0142984, average accuracy 0.997796,
Train epoch 40, average loss 0.0110543, average accuracy 0.998998,
	Dev epoch 40, average loss 0.478341, average accuracy 0.884171,
		Time taken for 40 epochs =  1374.086744070053
Train epoch 42, average loss 0.012298, average accuracy 0.998397,
Train epoch 44, average loss 0.0116889, average accuracy 0.998197,
	Dev epoch 45, average loss 0.448394, average accuracy 0.88587,
Train epoch 46, average loss 0.0107089, average accuracy 0.998197,
Train epoch 48, average loss 0.00953887, average accuracy 0.998898,
Train epoch 50, average loss 0.0097256, average accuracy 0.998898,
	Dev epoch 50, average loss 0.424627, average accuracy 0.886209,
		Time taken for 50 epochs =  1708.131004333496
Train epoch 52, average loss 0.00792942, average accuracy 0.999099,
Train epoch 54, average loss 0.00777054, average accuracy 0.999099,
	Dev epoch 55, average loss 0.434766, average accuracy 0.887228,
Train epoch 56, average loss 0.00812112, average accuracy 0.999099,
Train epoch 58, average loss 0.00817043, average accuracy 0.998798,
Train epoch 60, average loss 0.00776972, average accuracy 0.998498,
	Dev epoch 60, average loss 0.447535, average accuracy 0.886889,
		Time taken for 60 epochs =  2042.1303217411041
Train epoch 62, average loss 0.00759579, average accuracy 0.998998,
Train epoch 64, average loss 0.00697335, average accuracy 0.998798,
	Dev epoch 65, average loss 0.514295, average accuracy 0.88519,
Train epoch 66, average loss 0.00579109, average accuracy 0.999199,
Train epoch 68, average loss 0.00583337, average accuracy 0.999499,

#### Changes. set trainable = True in glove embeddings. Changed learning rate back to 0.01 initial.
# batches = 156
Train epoch 0, average loss 0.399899, average accuracy 0.858273,
	Dev epoch 0, average loss 0.374428, average accuracy 0.857337,
		Time taken for 0 epochs =  34.83389401435852
Train epoch 2, average loss 0.326706, average accuracy 0.869391,
Train epoch 4, average loss 0.269826, average accuracy 0.891126,
	Dev epoch 5, average loss 0.303238, average accuracy 0.88519,
Train epoch 6, average loss 0.219368, average accuracy 0.911659,
Train epoch 8, average loss 0.171234, average accuracy 0.935296,
Train epoch 10, average loss 0.13296, average accuracy 0.953325,
	Dev epoch 10, average loss 0.293018, average accuracy 0.887568,
		Time taken for 10 epochs =  370.47870922088623
Train epoch 12, average loss 0.100562, average accuracy 0.967348,
Train epoch 14, average loss 0.0793127, average accuracy 0.977063,
	Dev epoch 15, average loss 0.320119, average accuracy 0.886209,
Train epoch 16, average loss 0.0582729, average accuracy 0.988482,
Train epoch 18, average loss 0.0456755, average accuracy 0.990385,
Train epoch 20, average loss 0.0405185, average accuracy 0.992788,
	Dev epoch 20, average loss 0.321604, average accuracy 0.886889,
		Time taken for 20 epochs =  705.0958936214447
Train epoch 22, average loss 0.0351258, average accuracy 0.993089,
Train epoch 24, average loss 0.0270392, average accuracy 0.996194,
	Dev epoch 25, average loss 0.399808, average accuracy 0.884171,
Train epoch 26, average loss 0.0262923, average accuracy 0.995994,
Train epoch 28, average loss 0.0242657, average accuracy 0.995994,
Train epoch 30, average loss 0.0208821, average accuracy 0.996394,
	Dev epoch 30, average loss 0.413923, average accuracy 0.886889,
		Time taken for 30 epochs =  1039.4113600254059
Train epoch 32, average loss 0.017492, average accuracy 0.997696,
Train epoch 34, average loss 0.0146527, average accuracy 0.998097,
	Dev epoch 35, average loss 0.386267, average accuracy 0.884851,
Train epoch 36, average loss 0.0168233, average accuracy 0.997396,
Train epoch 38, average loss 0.0142984, average accuracy 0.997796,
Train epoch 40, average loss 0.0110543, average accuracy 0.998998,
	Dev epoch 40, average loss 0.478341, average accuracy 0.884171,
		Time taken for 40 epochs =  1374.086744070053
Train epoch 42, average loss 0.012298, average accuracy 0.998397,
Train epoch 44, average loss 0.0116889, average accuracy 0.998197,
	Dev epoch 45, average loss 0.448394, average accuracy 0.88587,
Train epoch 46, average loss 0.0107089, average accuracy 0.998197,
Train epoch 48, average loss 0.00953887, average accuracy 0.998898,
Train epoch 50, average loss 0.0097256, average accuracy 0.998898,
	Dev epoch 50, average loss 0.424627, average accuracy 0.886209,
		Time taken for 50 epochs =  1708.131004333496
Train epoch 52, average loss 0.00792942, average accuracy 0.999099,
Train epoch 54, average loss 0.00777054, average accuracy 0.999099,
	Dev epoch 55, average loss 0.434766, average accuracy 0.887228,
Train epoch 56, average loss 0.00812112, average accuracy 0.999099,
Train epoch 58, average loss 0.00817043, average accuracy 0.998798,
Train epoch 60, average loss 0.00776972, average accuracy 0.998498,
	Dev epoch 60, average loss 0.447535, average accuracy 0.886889,
		Time taken for 60 epochs =  2042.1303217411041
Train epoch 62, average loss 0.00759579, average accuracy 0.998998,
Train epoch 64, average loss 0.00697335, average accuracy 0.998798,
	Dev epoch 65, average loss 0.514295, average accuracy 0.88519,
Train epoch 66, average loss 0.00579109, average accuracy 0.999199,
Train epoch 68, average loss 0.00583337, average accuracy 0.999499,